## Custom Agent

In [106]:
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent
from langchain.utilities import SerpAPIWrapper
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.chat_models import ChatOpenAI

In [107]:
from getpass import getpass
SERPAPI_API_KEY = getpass()

In [108]:
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
        return_direct=True,
    ),
    Tool(
        name="LLM",
        func=search.run,
        description="useful for when you need to answer questions about current events",
        return_direct=True,
    ),
]

In [109]:
class FakeAgent(BaseSingleActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        chat = ChatOpenAI()
        chat_response = chat([HumanMessage(content=f'''Respond with either 'Yes' or 'No'. 
                                           Should we use a Google search tool, given the following user query: {kwargs['input']}
                                           Only reply with 'Yes' or 'No'.
                                           Favor using tools when dates and times are concerned.
                                           ''')])
        # If yes:
        if 'yes'.lower() in chat_response.content.lower():
            return AgentAction(tool="Search", tool_input=kwargs["input"], log="")
        else:
            # Reply in this section with a different tool / LLM to use:
            chat_response = chat([HumanMessage(content=f'''Act as an AI assistant and respond effectively to the user's query: {kwargs['input']}''')])
            return AgentFinish(return_values={"output": chat_response.content}, log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        raise NotImplementedError

In [110]:
agent = FakeAgent()
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True,
)

agent_executor.run("My name is Jane!")



> Entering new AgentExecutor chain...


> Finished chain.


'Hello Jane! How can I assist you today?'

In [111]:
agent_executor.run("Research data engineering principles and practices.")



> Entering new AgentExecutor chain...


> Finished chain.


'Data engineering is a field that focuses on designing and implementing systems to collect, store, and process large amounts of data. It involves various principles and practices to ensure efficient and reliable data management. Here are some key aspects of data engineering principles and practices:\n\n1. Data Collection: Data engineers work on establishing mechanisms to collect data from various sources, such as databases, APIs, logs, and sensors. They ensure that data is extracted accurately and in a timely manner.\n\n2. Data Storage: Data engineers design storage systems capable of handling large volumes of data. They utilize technologies like databases, data lakes, and data warehouses to store structured, semi-structured, and unstructured data.\n\n3. Data Processing: Data engineers develop processes to transform and clean the collected data. This involves activities like data validation, normalization, aggregation, and deduplication. They also employ techniques like data quality ch

----------------------------------------

In [112]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.utilities import SerpAPIWrapper
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, SystemMessage
import re

In [113]:
# Define which tools the agent can use to answer user queries
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [114]:
# Set up the base template
template = """
Act as a personal assistant, you are tasked with conducting an interview with the user.

If you don't need to use a reply, you must follow this response format:

Reply: ...A reply to the user...

If you need to use a tool, to answer the question, you have access to the following tools:

{tools}

Whenever you need to use a tool, you must follow pattern:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

---

To summarize the response formats must either be:
- Final Answer: ...
- Reply: ...

You must always use one of the above response formats!
---

Previous Chat History:
Chat history {chat_history}

These were previous tasks you completed:

Begin!

Question: {input}
{agent_scratchpad}

"""

In [117]:
from typing import Any

# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool] 
    memory: Any

    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [SystemMessage(content=formatted)]

In [118]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=False, input_key="input")

In [119]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Reply:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Conversation:")[-1].strip()},
                log=llm_output,
            )

        # Check if agent should finish
        elif "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [120]:
output_parser = CustomOutputParser()
llm = ChatOpenAI(temperature=0, model='gpt-4')

In [121]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "chat_history"]
)

In [122]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

In [123]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
    memory=memory
)

In [124]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools,  memory=memory)

In [126]:
agent_executor.run({
    "input": "My name is James",
})

'Reply: Nice to meet you, James! How can I assist you today?'

In [127]:
agent_executor.run({
    "input": "What is my name?",
})

'Your name is James.'

In [128]:
agent_executor.run({
    "input": "What is the average age in the US?",
})

'The average age in the United States is 38.9 years.'